In [1]:
from calliope.data import DataManager
from calliope.market import MeritOrderNEM, DataToBidObjInterface
import pandas as pd
import sqlite3
con = sqlite3.connect("E:\Code\Calliope/notebooks\historical_mms.db")
import datetime
data_manager = DataManager(con)
import matplotlib.pyplot as plt

start_date = '2019/03/01 00:00:00'
end_date = '2019/04/01 00:00:00'
region = 'NSW1'

data_manager.load_data(start_date, end_date, region)

In [2]:
# Testing new market model
from calliope.defaults import *
from calliope import market_model

In [ ]:
nem = MeritOrderNEM(data_manager)
nem.build()
nem.load_bids_from_pickle(['bid_dict_mar_nsw.pkl'])

In [3]:
di_curr = '2019-03-25 16:05:00'
settlementdate = '2019-03-25'

In [4]:
price_bids =data_manager.price_bids.loc[data_manager.price_bids.SETTLEMENTDATE == settlementdate]
volume_bids =data_manager.volume_bids.loc[data_manager.volume_bids.INTERVAL_DATETIME == di_curr]
volume_bids.loc[:, VOLUME_BID_COLS] = volume_bids[VOLUME_BID_COLS].cumsum(axis=1)

# map markets to duids
mapping = dict(volume_bids.groupby('BIDTYPE')['DUID'].unique().reset_index().values)

# get max number of units
n_units = max(mapping[k].shape[0] for k in mapping.keys())

def get_vol_bid(m, unit, band):
    try: 
        return volume_bids.loc[
        (volume_bids.BIDTYPE == MARKETS[m]) & 
        (volume_bids.DUID == mapping[MARKETS[m]][unit])
        , f'BANDAVAIL{band+1}'
    ].iloc[0]
    except:
        return 0
    
def get_maxavail(m, unit):
    try: 
        return volume_bids.loc[
        (volume_bids.BIDTYPE == MARKETS[m]) & 
        (volume_bids.DUID == mapping[MARKETS[m]][unit])
        , 'MAXAVAIL'
    ].iloc[0]
    except:
        return 0
    
def get_price_bid(m, unit, band):
    try: 
        return price_bids.loc[
        (price_bids.BIDTYPE == MARKETS[m]) & 
        (price_bids.DUID == mapping[MARKETS[m]][unit])
        , f'PRICEBAND{band+1}'
    ].iloc[0]
    except:
        return 0
    
price_bid_dict = {} 
for m in range(len(MARKETS)):
    for unit in range(n_units):
        for band in range(10):
            key =   (m, unit, band)
            price_bid_dict[key] = get_price_bid(m, unit, band) 

vol_bid_dict = {} 
for m in range(len(MARKETS)):
    for unit in range(n_units):
        for band in range(10):
            key =   (m, unit, band)
            vol_bid_dict[key] = get_vol_bid(m, unit, band) 

max_avail = {} 
for m in range(len(MARKETS)):
    for unit in range(n_units):
            key =   (m, unit)
            max_avail[key] = get_maxavail(m, unit) 

# Create dict here bc lazy
fcas_req = data_manager.fcas_requirements
demand = data_manager.regional_demand

# absolute shambles of data management here... but it workds
demand_val = demand.loc[demand.SETTLEMENTDATE == di_curr, 'TOTALDEMAND'].reset_index(drop=True).iloc[0]
demand_dict = fcas_req.loc[fcas_req.SETTLEMENTDATE == di_curr, fcas_req.columns!='SETTLEMENTDATE'].to_dict(orient='records')[0]
demand_dict['ENERGY'] = demand_val

In [493]:
nem.set_current_dispatch_interval(di_curr)
nem.add_bids()
dispatch = nem.dispatch()
nem.step()

In [519]:
# warm start the MIP using the merit order.
# this can be preprocessed along with the above data not accounting for BESS decisions
warm_start={}

# dispatch['ENERGY']['dispatch']

# fill UnitEnabled
warm_start['UnitEnabled'] = {}
warm_start['EnabledCapacity'] = {}
warm_start['BidCapacity'] = {}
warm_start['mcp'] = {}

for m, m_name in zip(range(len(MARKETS)), MARKETS):
    warm_start['UnitEnabled'][m] = {}
    warm_start['EnabledCapacity'][m] = {}
    warm_start['BidCapacity'][m] = {}

    try:
        warm_start['mcp'][m] = dispatch[m]['price']
    except KeyError:
        warm_start['mcp'][m] = 0.0


    # warm start with the following variables
    for i in range(n_units):
        try:
            unit_i = mapping[m_name][i]
        except IndexError:
            warm_start['UnitEnabled'][m][i] = 0
            warm_start['EnabledCapacity'][m][i] = 0.0
            warm_start['BidCapacity'][m][i] = 0.0
        try:
            warm_start['UnitEnabled'][m][i] = int(dispatch[m_name]['dispatch'][unit_i] > 0.0)
        except KeyError:
            warm_start['UnitEnabled'][m][i] = 0
        try:
            warm_start['EnabledCapacity'][m][i] = dispatch[m_name]['dispatch'][unit_i]
        except KeyError:
            warm_start['EnabledCapacity'][m][i] = 0.0
        try:
            warm_start['BidCapacity'][m][i] = dispatch[m_name]['dispatch'][unit_i]
        except KeyError:
            warm_start['BidCapacity'][m][i] = 0.0




In [33]:
import importlib
importlib.reload(market_model)

<module 'calliope.market_model' from 'E:\\Code\\Calliope\\calliope\\market_model.py'>

In [34]:
model = market_model.MarketClearingModel_AdjustedIndexer(
    mapping, 
    demand_dict, 
    price_bid_dict, 
    vol_bid_dict,
    max_avail
)

Set parameter Presolve to value 1
Set parameter MIPGap to value 0.05
Set parameter MIPFocus to value 2
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 10.0 (19043.2))

CPU model: AMD Ryzen 7 5700X 8-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 3369 rows, 8109 columns and 10571 nonzeros
Model fingerprint: 0x5ee07765
Model has 1080 quadratic constraints
Variable types: 1629 continuous, 6480 integer (6480 binary)
Coefficient statistics:
  Matrix range     [1e-02, 1e+04]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+06]
  Objective range  [9e+00, 9e+03]
  Bounds range     [1e+00, 2e+04]
  RHS range        [1e+00, 9e+03]
  QRHS range       [1e+06, 1e+06]
Presolve removed 2976 rows and 6821 columns
Presolve time: 0.02s
Presolved: 695 rows, 1571 columns, 3813 nonzeros
Presolved model has 176 SOS constraint(s)
Variable types: 372 continuous, 1199 integer (

In [36]:
for m in MARKETS:
    print(f"{m}: {model.getVarByName(f'mcp_{m}').X }")


ENERGY: 104.0
RAISEREG: 2.84
RAISE6SEC: 1.74
RAISE60SEC: 0.01
RAISE5MIN: 2.8
LOWERREG: 24.979999999993016
LOWER6SEC: 0.0
LOWER60SEC: 0.04
LOWER5MIN: 2.8


: 

In [487]:
for i in range(len(mapping['ENERGY'])):
    print(f"{mapping['ENERGY'][i]}:\n Accepted bid <p,q>: {model.getVarByName(f'BidPrice[0,{i}]').X}, {model.getVarByName(f'BidCapacity[0,{i}]').X}")


BLOWERNG:
 Accepted bid <p,q>: 0.0, 0.0
BOCORWF1:
 Accepted bid <p,q>: 0.0, 113.0
BODWF1:
 Accepted bid <p,q>: -981.9, 111.0
BROKENH1:
 Accepted bid <p,q>: 0.0, 53.0
BW01:
 Accepted bid <p,q>: 140.0, 660.0
BW02:
 Accepted bid <p,q>: 50.09, 610.0
BW03:
 Accepted bid <p,q>: 0.0, 350.0
BW04:
 Accepted bid <p,q>: 87.77, 640.0
CG1:
 Accepted bid <p,q>: 0.0, 0.0
CG2:
 Accepted bid <p,q>: 0.0, 0.0
CG3:
 Accepted bid <p,q>: 0.0, 0.0
CG4:
 Accepted bid <p,q>: 0.0, 0.0
COLEASF1:
 Accepted bid <p,q>: 0.0, 150.0
CROOKWF2:
 Accepted bid <p,q>: -986.6, 91.0
DG_NSW1:
 Accepted bid <p,q>: 0.0, 0.0
ER01:
 Accepted bid <p,q>: 58.51, 680.0
ER02:
 Accepted bid <p,q>: 25.080000000000002, 650.0
ER03:
 Accepted bid <p,q>: 48.27, 580.0
ER04:
 Accepted bid <p,q>: 0.0, 0.0
GULLRSF1:
 Accepted bid <p,q>: 0.0, 10.0
GULLRWF1:
 Accepted bid <p,q>: 0.0, 165.0
GUNNING1:
 Accepted bid <p,q>: 0.0, 47.0
GUTHEGA:
 Accepted bid <p,q>: 0.0, 0.0
HUMENSW:
 Accepted bid <p,q>: 0.0, 0.0
HVGTS:
 Accepted bid <p,q>: 0.0, 0.0
LD0